In [43]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from keras.optimizers import Adam

In [17]:
data = pd.read_csv('house_prices.csv')

In [18]:
data

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [19]:
features = data.drop(['price'], axis=1).values
target = data['price'].values

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler.fit_transform(features)

ValueError: could not convert string to float: 'yes'

In [9]:
categorical_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']
numerical_cols = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

In [21]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ], remainder='passthrough')


In [23]:
features = pd.DataFrame(features, columns=data.columns.drop('price'))

In [24]:
features_processed = preprocessor.fit_transform(features)

In [25]:
features_processed_df = pd.DataFrame(features_processed)

In [26]:
target_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaled = target_scaler.fit_transform(target.reshape(-1, 1))

In [27]:
def create_dataset(features, target, time_step=1):
    X, Y = [], []
    for i in range(len(features) - time_step - 1):
        a = features[i:(i + time_step), :]
        X.append(a)
        Y.append(target[i + time_step])
    return np.array(X), np.array(Y)

In [29]:
time_step = 10
X, y = create_dataset(features_processed_df.values, target_scaled, time_step)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(time_step, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=100, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=1))

In [44]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer= optimizer, loss='mean_squared_error')

In [54]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
14/14 [==============================] - 0s 25ms/step - loss: 0.0011 - val_loss: 8.7730e-04
Epoch 2/100
14/14 [==============================] - 0s 22ms/step - loss: 0.0012 - val_loss: 9.9503e-04
Epoch 3/100
14/14 [==============================] - 0s 23ms/step - loss: 0.0012 - val_loss: 9.1399e-04
Epoch 4/100
14/14 [==============================] - 0s 30ms/step - loss: 9.1704e-04 - val_loss: 5.9981e-04
Epoch 5/100
14/14 [==============================] - 0s 27ms/step - loss: 0.0011 - val_loss: 6.8750e-04
Epoch 6/100
14/14 [==============================] - 0s 32ms/step - loss: 9.3597e-04 - val_loss: 0.0013
Epoch 7/100
14/14 [==============================] - 0s 28ms/step - loss: 8.2895e-04 - val_loss: 7.9309e-04
Epoch 8/100
14/14 [==============================] - 0s 34ms/step - loss: 8.6375e-04 - val_loss: 0.0013
Epoch 9/100
14/14 [==============================] - 0s 27ms/step - loss: 0.0012 - val_loss: 5.9915e-04
Epoch 10/100
14/14 [==============================] - 0s

In [46]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

4/4 [==============================] - 0s 10ms/step


In [47]:
train_predict = target_scaler.inverse_transform(train_predict)
test_predict = target_scaler.inverse_transform(test_predict)
y_train_actual = target_scaler.inverse_transform(y_train)
y_test_actual = target_scaler.inverse_transform(y_test)

In [48]:
train_rmse = np.sqrt(np.mean((train_predict - y_train_actual) ** 2))
test_rmse = np.sqrt(np.mean((test_predict - y_test_actual) ** 2))

In [49]:
print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')

Train RMSE: 323890.9784194323
Test RMSE: 351551.98867017037


In [52]:
train_mse = np.mean((train_predict - y_train_actual) ** 2)
test_mse = np.mean((test_predict - y_test_actual) ** 2)